In [4]:
import os, base64, sys
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

KEY=os.urandom(16)
IV=os.urandom(16)

def encrypt(msg):
    msg = pad(msg,16)
    cipher = AES.new(KEY,AES.MODE_CBC,IV)
    encrypted = cipher.encrypt(msg)
    encrypted = encrypted.hex()
    msg = IV.hex() + encrypted
    return msg

def decrypt(msg,iv):
    if len(msg) > 16:
        print("Message must be <= 16")
    
    
    decrypted = unpad(cipher.decrypt(msg),16).decode()
    return decrypted

def weirdify(inp):
    iv = bytes.fromhex(inp[:32])
    msg = bytes.fromhex(inp[32:])
    command = decrypt(msg,iv)
    return command

In [51]:
def crack():
    return "§"

eval('crack()      ')

'§'

# Attack

Malleability of CBC, where we have to change the plaintext of the first block such that `crack()` will be executed

In [54]:
# We need the same size because of padding
plaintext = b'trapped_forever'
chosen_pt = b'crack()'
chosen_pt = chosen_pt + b" " * (15 - len(chosen_pt))
len(plaintext)

15

In [56]:
ciphertext = bytes.fromhex("cc155af266afb52379e2f8b66a6d1164853325783ecb7fd32c3c91ee0f5b2c63")
iv,msg = ciphertext[:16],ciphertext[16:]
len(iv),len(msg)

(16, 16)

### We have after encryption with padding 

$m = $`trapped_forever\x0e`

$m' = $`print(crack()) \0xe`

$c = Enc(k,m \oplus IV)$

#### and decryption is

$m = Dec(c) \oplus IV$

Therefore, 

$m' = Dec(c) \oplus IV \oplus m \oplus m'$ such that we set our new iv to $IV \oplus m \oplus m'$

In [57]:
def xor(a,b):
    return bytes([x^y for x,y in zip(a,b)])

In [58]:
padded_plaintext = pad(plaintext,16)
padded_chosentext = pad(chosen_pt,16)

assert padded_chosentext[-1] == padded_plaintext[-1]
padded_chosentext

b'crack()        \x01'

In [59]:
iv_mal = xor(iv,xor(padded_plaintext,padded_chosentext))

(iv_mal + msg).hex()

'db155ae17de2f85c3fadaaf33c284364853325783ecb7fd32c3c91ee0f5b2c63'

## Building a oracle

In [61]:
from pwn import *

def xor(a,b):
    return bytes([x^y for x,y in zip(a,b)])

In [137]:
r = remote("jail-crypto.challenge.cryptonite.team","1337")

def query(command,second_cmd):
    
    plaintext = b'trapped_forever'
    chosen_pt = command + b" " * (15 - len(command))
    
    padded_plaintext = pad(plaintext,16)
    padded_chosentext = pad(chosen_pt,16)

    assert padded_chosentext[-1] == padded_plaintext[-1]

    r.recvuntil(b"As a gift we'll give you a sample encryption\n")

    ciphertext = bytes.fromhex(r.recvuntil(b"\n").decode("ASCII"))
    r.recvuntil(b">>")

    iv,msg = ciphertext[:16],ciphertext[16:]

    iv_mal = xor(iv,xor(padded_plaintext,padded_chosentext))
    payload = (iv_mal + msg).hex()

    r.sendline(bytes(payload,"ASCII"))
    r.sendline(second_cmd + b"\n")
    print(r.recvall(timeout=5).decode("ASCII"))

query(b"exec(input(''))",b"import os;os.system('ls -la');print('test')")

[x] Opening connection to jail-crypto.challenge.cryptonite.team on port 1337
[x] Opening connection to jail-crypto.challenge.cryptonite.team on port 1337: Trying 34.91.213.62
[+] Opening connection to jail-crypto.challenge.cryptonite.team on port 1337: Done
[x] Receiving all data
[x] Receiving all data: 0B
[x] Receiving all data: 287B
[x] Receiving all data: 294B
[+] Receiving all data: Done (294B)
[*] Closed connection to jail-crypto.challenge.cryptonite.team port 1337
total 16
drwxr-xr-x 2 nobody nogroup 4096 Dec 10 13:44 .
drwxr-xr-x 3 nobody nogroup 4096 Dec 10 13:44 ..
-rw-r--r-- 1 nobody nogroup   18 Dec  6 13:08 flag.txt
-rw-r--r-- 1 nobody nogroup    0 Dec  6 13:08 nite{Th3__gr3at_esc4p3}
-rwxr-xr-x 1 nobody nogroup 1387 Dec  6 13:08 server.py
test
>>
